In [1]:
import numpy as np
import pandas as pd
from keras import layers,models
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
train=pd.read_csv('../input/train.csv')
test=pd.read_csv('../input/test.csv')

In [3]:
x=train.drop('label',axis=1)
x_test=test.copy()
x=x/255
x_test=x_test/255


In [4]:
y=train['label']
y=pd.Categorical(y)

In [5]:
y=pd.get_dummies(y)

In [6]:
x_train,x_val,y_train,y_val=train_test_split(x,y,test_size=0.1,random_state=0)

In [7]:
x_train=x_train.values
y_train=y_train.values
x_val=x_val.values
y_val=y_val.values
x_test=x_test.values

In [8]:
x_train=x_train.reshape(37800,28,28,1)
x_val=x_val.reshape(4200,28,28,1)
x_test=x_test.reshape(28000,28,28,1)

In [9]:
model= models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape = (28,28,1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.Dropout(0.25))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [10]:
model.add(layers.Flatten())
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(10,activation='softmax'))

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
dropout_1 (Dropout)          (None, 3, 3, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
__________

In [12]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [13]:
model.fit(x_train, y_train,
          batch_size=64,
          epochs=15,
          verbose=1,
          validation_data=(x_val, y_val))


Instructions for updating:
Use tf.cast instead.
Train on 37800 samples, validate on 4200 samples
Epoch 1/15
37800/37800 [==============================] - 6s 156us/step - loss: 0.2358 - acc: 0.9256 - val_loss: 0.0857 - val_acc: 0.9736
Epoch 2/15
37800/37800 [==============================] - 4s 105us/step - loss: 0.0671 - acc: 0.9784 - val_loss: 0.0696 - val_acc: 0.9781
Epoch 3/15
37800/37800 [==============================] - 4s 105us/step - loss: 0.0485 - acc: 0.9847 - val_loss: 0.0527 - val_acc: 0.9826
Epoch 4/15
37800/37800 [==============================] - 4s 105us/step - loss: 0.0374 - acc: 0.9882 - val_loss: 0.0573 - val_acc: 0.9824
Epoch 5/15
37800/37800 [==============================] - 4s 106us/step - loss: 0.0316 - acc: 0.9902 - val_loss: 0.0327 - val_acc: 0.9886
Epoch 6/15
37800/37800 [==============================] - 4s 105us/step - loss: 0.0254 - acc: 0.9924 - val_loss: 0.0460 - val_acc: 0.9876
Epoch 7/15
37800/37800 [==============================] - 4s 105us/step - l

In [14]:
sub=pd.read_csv('../input/sample_submission.csv')

In [15]:
pred=model.predict(x_test,verbose=0)
new_pred = [np.argmax(y, axis=None, out=None) for y in pred]
output=pd.DataFrame({'ImageId':sub['ImageId'],'Label':new_pred})
output.to_csv('Digit_recognizer.csv', index=False)